In [1]:
import cv2
import os
import numpy as np
import time
from skimage.measure import label
import matplotlib.pyplot as plt
import numba

# 适用于图像边缘检测的评价方法
## Figure of Merit 品质因数 F值
\begin{equation}
F = \frac{1}{max(N_{GT}, N_{Seg})}\sum_{i=1}^{N_{Seg}}\frac{1}{1 + \xi \times d_i^2}
\end{equation}

$N_{GT}$为人工检视标注得到的真实的边缘像素点个数 ，$N_{Seg}$ 为通过检测算法检测的边缘像素点个数, $d_i$为检测算法得到的第 i 个边缘像素点和离它最近的实际边缘像素点间的欧式距离，$\xi$ 为常量系数以惩罚错位的边缘，该系数一般为0.1。F值越大表明检测出的边缘越靠近人工标注的实际边缘, 该边缘检测算法的检测精度越高

参考：Abdou I E, Pratt W K. Quantitative design and evaluation of enhancement thresholding edge detectors[J]. Proceedings of the IEEE, 1979, 67(5): 753-763

In [2]:
def get_figure_of_merit(pred, mask, const_index = 0.1):
    """
    针对真值图 mask 和预测图pred计算F值并返回
    针对pred中的每个点，遍历其60邻域内最近的mask点,计算d_i,最终计算F_score
    本方法速度最慢
    :param pred: 预测图，[0,255]，背景为0，前景为255
    :param mask: 真值图，[0,255]，背景为0，前景为255
    :return: f_score
    """
    num_pred = np.count_nonzero(pred[pred == 255])
    num_mask = np.count_nonzero(mask[mask == 255])
    num_max = num_pred if num_pred > num_mask else num_mask
    temp = 0.0
    for index_x in range(0, pred.shape[0]):
        for index_y in range(0, pred.shape[1]):
            if pred[index_x, index_y] == 255:
                distance = get_dis_from_mask_point(
                    mask, index_x, index_y)
                temp = temp + 1 / (1 + const_index * pow(distance, 2))
    f_score = (1.0 / num_max) * temp
    return f_score

def get_dis_from_mask_point(self, mask, index_x, index_y, neighbor_length=60):
    """
    计算检测到的边缘点与离它最近边缘点的距离
    """

    if mask[index_x, index_y] == 255:
        return 0
    distance = neighbor_length / 2
    region_start_row = 0
    region_start_col = 0
    region_end_row = mask.shape[0]
    region_end_col = mask.shape[1]
    if index_x - neighbor_length > 0:
        region_start_row = index_x - neighbor_length
    if index_x + neighbor_length < mask.shape[0]:
        region_end_row = index_x + neighbor_length
    if index_y - neighbor_length > 0:
        region_start_col = index_y - neighbor_length
    if index_y + neighbor_length < mask.shape[1]:
        region_end_col = index_y + neighbor_length
        # Get the corrdinate of mask in neighbor region
        # becuase the corrdinate will be chaneged after slice operation, we add it manually
    x, y = np.where(mask[region_start_row: region_end_row, region_start_col: region_end_col] == 255)
        
    min_distance = np.amin(np.linalg.norm(np.array([x + region_start_row,y + region_start_col]) - np.array([[index_x], [index_y]]), axis=0))
        
    return min_distance

# 适用于图像边缘检测的评价方法
## F1_measure_for_boundary

定义边缘可移动的范围为$\varepsilon$，这个范围一般为边缘细化前的宽度，通常设置为2，统计落在mask边缘此距离内的pred点数目为TP，统计没有落在mask边缘此距离内的pred点数目为FP. 统计mask中没有落到pred边缘此距离范围内的为FN
\begin{equation}
Precision(P) = \frac{TP}{TP + TN}
\end{equation}

\begin{equation}
Recall(R) = \frac{TP}{TP + FN}
\end{equation}

\begin{equation}
F1 = \frac{2 \times P \times R}{P + R}
\end{equation}
![F1_for_boundary](img/F1_score_in_boundary.png)

参考：Waggoner J , Zhou Y , Simmons J , et al. 3D Materials Image Segmentation by 2D Propagation: A Graph-Cut Approach Considering Homomorphism[J]. IEEE Transactions on Image Processing, 2013, 22(12):5282-5293.

In [3]:
def get_f1_measure_for_boundary(pred, mask, varepsilon = 2):
    """
    针对真值图 mask 和预测图 pred 计算F1值并返回,仅适用于边缘检测
    本方法实现了F1值的计算：
    引用自:Waggoner J , Zhou Y , Simmons J , et al. 3D Materials Image Segmentation by 2D Propagation: A Graph-Cut Approach Considering Homomorphism[J]. 
    IEEE Transactions on Image Processing, 2013, 22(12):5282-5293.
    先分别计算 pred 和 mask 的坐标x_pred, y_pred, x_mask, y_mask, 假设pred中有m个点，mask中有n个点
    先建成（2，m, n）的矩阵，第一行表示x，第二行表示y, 然后分别是两个方向上，pred减mask的平方
    再将两行加起来形成（m,n），再在axis=1上排序，第0位置即为pred中每个点对应mask中最近点的平方,即可求f_score
    该方法由于要生成（2，m, n）三维矩阵，空间消耗巨大，且随着图像变大指数增长，运行时需考虑图像大小
    :param pred: 预测图，[0,255]，背景为0，前景为255
    :param mask: 真值图，[0,255]，背景为0，前景为255
    :return: f_score
    """
    num_pred = np.count_nonzero(pred[pred == 255])
    num_mask = np.count_nonzero(mask[mask == 255])
    TP = 0
    FP = 0
    FN = 0
    x_pred, y_pred = np.where(pred == 255)
    x_mask, y_mask = np.where(mask == 255)
    dis_xy = np.zeros((2, num_pred, num_mask))
    for index in range(0, num_pred):
        dis_xy[0, index, :] = np.power(x_mask - x_pred[index], 2)
        dis_xy[1, index, :] = np.power(y_mask - y_pred[index], 2)
    dis = np.sqrt(dis_xy[0, :, :] + dis_xy[1, :, :])
    dis.sort(axis=1)
    
    TP = np.count_nonzero(dis[:, 0] < varepsilon)
    FP = num_pred - TP
    FN = num_mask - TP
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1_for_boundary = 2 * precision / (precision + recall)
    return f1_for_boundary

# 适用于图像分割的评价方法
## Pixel accuracy
\begin{equation}
\frac{\sum_{i}n_{ii}}{\sum_{i}t_i}
\end{equation}
## Mean accuracy
\begin{equation}
\frac{1}{n_{cl}} \times \sum_{i}\frac{n_{ii}}{t_i}
\end{equation}
## Mean IOU
\begin{equation}
\frac{1}{n_{cl}} \times \sum_{i}\frac{n_{ii}}{t_i + \sum_{j}n_{ji} - n_{ii}}
\end{equation}

上述方法为 Pascal Visual Object Classes (VOC) Challenge 、CitySpaces的语义分割部分的标准衡量指标，在原论文中表示为$Seg_{accuracy} = \frac{TP}{TP + FP + FN}$

## Frequency weighted IOU
\begin{equation}
\frac{1}{\sum_{k}t_k} \times \sum_{i}\frac{t_i \times n_{ii}}{t_i + \sum_{j}n_{ji} - n_{ii}}
\end{equation}

$n_{ij}$是像素原本为类 i 被分为类 j 的数目，一共有$n_{cl}$个类，$t_i=\sum_{j}n_{ij}$是类 i 的所有像素个数

参考：
Long J , Shelhamer E , Darrell T . Fully Convolutional Networks for Semantic Segmentation[J]. IEEE Transactions on Pattern Analysis & Machine Intelligence, 2014, 39(4):640-651.

Everingham M, Eslami S M A, Van Gool L, et al. The pascal visual object classes challenge: A retrospective[J]. International journal of computer vision, 2015, 111(1): 98-136.

Cordts M, Omran M, Ramos S, et al. The cityscapes dataset for semantic urban scene understanding[C]//Proceedings of the IEEE conference on computer vision and pattern recognition. 2016: 3213-3223.

In [4]:
def get_pixel_accuracy(pred, mask, n_cl=2):
    """
    针对真值图 mask 和预测图 pred 计算 pixel accuracy
    评估方法参照FCN 论文：
    Long J , Shelhamer E , Darrell T . Fully Convolutional Networks for Semantic Segmentation[J]. 
    IEEE Transactions on Pattern Analysis & Machine Intelligence, 2014, 39(4):640-651.
    :param pred: 预测图
    :param mask: 真值图
    :param n_cl: 一共多少个类别
    """
    if np.amax(mask) == 255 and n_cl == 2:
        pred = pred / 255
        mask = mask / 255
    temp_n_ii = 0
    temp_t_i = 0
    for i_cl in range(0, n_cl):
        temp_n_ii += np.count_nonzero(mask[pred == i_cl] == i_cl)
        temp_t_i  += np.count_nonzero(mask == i_cl)
    pixel_accuracy_score = temp_n_ii / temp_t_i
    return pixel_accuracy_score

def get_mean_accuracy(pred, mask, n_cl=2):
    """
    针对真值图 mask 和预测图 pred 计算 mean accuracy
    评估方法参照FCN 论文：
    Long J , Shelhamer E , Darrell T . Fully Convolutional Networks for Semantic Segmentation[J]. 
    IEEE Transactions on Pattern Analysis & Machine Intelligence, 2014, 39(4):640-651.
    :param pred: 预测图
    :param mask: 真值图
    :param n_cl: 一共多少个类别
    """
    if np.amax(mask) == 255 and n_cl == 2:
        pred = pred / 255
        mask = mask / 255
    temp = 0
    for i_cl in range(0, n_cl):
        n_ii = np.count_nonzero(mask[pred == i_cl] == i_cl)
        t_i = np.count_nonzero(mask == i_cl)
        temp += n_ii / t_i
    mean_accuracy_score = temp / n_cl
    return mean_accuracy_score

def get_m_iou_for_class(pred, mask, n_cl=2):
    """
    针对真值图 mask 和预测图 pred 针对每个类别计算mIoU,适用于图像分割，不适用于图像实例分割
    评估方法参照FCN 论文：
    Long J , Shelhamer E , Darrell T . Fully Convolutional Networks for Semantic Segmentation[J]. 
    IEEE Transactions on Pattern Analysis & Machine Intelligence, 2014, 39(4):640-651.
    :param pred: 预测图
    :param mask: 真值图
    :param n_cl: 一共多少个类别
    """
    if np.amax(mask) == 255 and n_cl == 2:
        pred = pred / 255
        mask = mask / 255
    temp = 0
    for i_cl in range(0, n_cl):
        n_ii = np.count_nonzero(mask[pred == i_cl] == i_cl)
        t_i = np.count_nonzero(mask == i_cl)
        temp += n_ii / (t_i + np.count_nonzero(pred == i_cl) - n_ii)
    fw_iou_for_class_score = temp / n_cl
    return mean_accuracy_score

def get_fw_iou_for_class(pred, mask, n_cl=2):
    """
    针对真值图 mask 和预测图 pred 针对每个类别计算fwIoU(Frequency weighted IOU),适用于图像分割，不适用于图像实例分割
    评估方法参照FCN 论文：
    Long J , Shelhamer E , Darrell T . Fully Convolutional Networks for Semantic Segmentation[J]. 
    IEEE Transactions on Pattern Analysis & Machine Intelligence, 2014, 39(4):640-651.
    :param pred: 预测图
    :param mask: 真值图
    :param n_cl: 一共多少个类别
    """
    if np.amax(mask) == 255 and n_cl == 2:
        pred = pred / 255
        mask = mask / 255
    temp_t_i = 0
    temp_iou = 0 
    for i_cl in range(0, n_cl):
        n_ii = np.count_nonzero(mask[pred == i_cl] == i_cl)
        t_i = np.count_nonzero(mask == i_cl)
        temp_iou += t_i * n_ii / (t_i + np.count_nonzero(pred == i_cl) - n_ii)
        temp_t_i += t_i
    fw_iou_for_class_score = temp_iou / temp_t_i
    return fw_accuracy_score

# 适用于图像实例分割的评价方法（特别针对材料科学）
## Cardinality difference
\begin{equation}
R = |G| - |S|
\end{equation}
$|G|$是真值图中的区域的个数， $|S|$ 是预测图中区域的个数，$G$为正数代表欠分割，$G$为负数过分割

参考：Waggoner J , Zhou Y , Simmons J , et al. 3D Materials Image Segmentation by 2D Propagation: A Graph-Cut Approach Considering Homomorphism[J]. IEEE Transactions on Image Processing, 2013, 22(12):5282-5293.

In [5]:
def get_cardinality_difference(pred, mask, target_image = 0):
    """
    针对真值图 mask 和预测图 pred 计算 cardinality_difference , 仅适用于材料或生物医学图像
    评估方法参照 
    Waggoner J , Zhou Y , Simmons J , et al. 3D Materials Image Segmentation by 2D Propagation: A Graph-Cut Approach Considering Homomorphism[J]. 
    IEEE Transactions on Image Processing, 2013, 22(12):5282-5293.
    :param pred: 预测图，[0, 255], 前景为255， 背景为0
    :param mask: 真值图，[0, 255]，前景为255， 背景为0
    :param target_image: 为0为纯铁晶粒图像，为1铝镧枝晶图像
    :return: map_score
    """
    if target_image == 0:
        # 预处理，得到的pred边缘结果图像在周围可能出现分割不良，手动补齐
        pred[0, :] = 255
        pred[:, 0] = 255
        pred[-1, :] = 255
        pred[:, -1] = 255
        mask[0, :] = 255
        mask[:, 0] = 255
        mask[-1, :] = 255
        mask[:, -1] = 255
        # 预处理，为保证按照每个实例进行识别，需要label,所以统一前背景
        mask = 255 - mask
        pred = 255 - pred
    label_mask, num_mask = label(mask, neighbors=4, background=0, return_num=True)
    label_pred, num_pred = label(pred, neighbors=4, background=0, return_num=True)
    cardinality_difference = num_mask - num_pred
    return cardinality_difference

# 适用于图像实例分割的评价方法
## Map_2018kdsb (map evaluated in 2018 data science bowl)
\begin{equation}
IoU(A,B) = \frac{A \cap B}{A \cup B}
\end{equation}

通过一系列IOU阈值评估结果，阈值从0.5增长到0.95以0.05的间隔增长：（0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95），在每个阈值上计算平均精确度后求和再平均。

举例：如果一个预测目标对一个真实结果的IOU超过0.5阈值，则认为正确匹配
\begin{equation}
\frac{TP(t)}{TP(t) + FP(t) + FN(t)}
\end{equation}

True positive（TP）：一个预测结果与一个真实结果的IOU>阈值

False positive（FP）：一个预测结果没有与之相对应的真实结果

False negative（FN）：一个真实结果没有与之相对应的预测结果

\begin{equation}
\frac{1}{|thresholds|}\sum_{t}\frac{TP(t)}{TP(t) + FP(t) + FN(t)}
\end{equation}

参考：https://www.kaggle.com/c/data-science-bowl-2018/overview/evaluation

In [6]:
def get_map_2018kdasb(pred, mask, target_image = 0):
    """
    针对真值图 mask 和预测图 pred 计算 map , 仅适用于二值图像
    评估方法参照2018 kaggle data science bowl: 
    https://www.kaggle.com/c/data-science-bowl-2018/overview/evaluation
    :param pred: 预测图，[0, 255], 前景为255， 背景为0
    :param mask: 真值图，[0, 255]，前景为255， 背景为0
    :param target_image: 为0为纯铁晶粒图像，为1铝镧枝晶图像
    :return: map_score
    """
    thresholds = np.array([0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95])
    tp = np.zeros(10)
    fp = np.zeros(10)
    fn = np.zeros(10)
    if target_image == 0:
        # 预处理，得到的pred边缘结果图像在周围可能出现分割不良，手动补齐
        pred[0, :] = 255
        pred[:, 0] = 255
        pred[-1, :] = 255
        pred[:, -1] = 255
        mask[0, :] = 255
        mask[:, 0] = 255
        mask[-1, :] = 255
        mask[:, -1] = 255
        # 预处理，为保证按照每个实例进行识别，需要label,所以统一前背景
        mask = 255 - mask
        pred = 255 - pred
        
    label_mask, num_mask = label(mask, neighbors=4, background=0, return_num=True)
    label_pred, num_pred = label(pred, neighbors=4, background=0, return_num=True)
    
    for i_pred in range(1, num_pred + 1):
        intersect_mask_labels = list(np.unique(label_mask[label_pred == i_pred]))   # 获得与之相交的所有label
        # 对与其相交的的所有mask label计算iou，后取其最值
        if 0 in intersect_mask_labels:
            intersect_mask_labels.remove(0)

        if len(intersect_mask_labels) == 0:   # 如果pred的某一个label没有与之对应的mask的label,则继续下一个label
            continue
        
        intersect_mask_label_area = np.zeros((len(intersect_mask_labels), 1))
        union_mask_label_area = np.zeros((len(intersect_mask_labels), 1))
        
        for index, i_mask in enumerate(intersect_mask_labels):
            intersect_mask_label_area[index, 0] = np.count_nonzero(label_pred[label_mask == i_mask] == i_pred)
            union_mask_label_area[index, 0] = np.count_nonzero((label_mask == i_mask) | (label_pred == i_pred))
        iou = intersect_mask_label_area / union_mask_label_area
        max_iou = np.max(iou, axis=0)
        # 根据最值将tp赋值
        # 此处基于一个重要理论：对于一个预测的晶粒，真实的晶粒有且仅有一个晶粒与其iou>0.5
        tp[thresholds < max_iou] = tp[thresholds < max_iou] + 1
    fp = num_pred - tp 
    fn = num_mask - tp
    map_score = np.average(tp/(tp + fp + fn))
    return map_score

In [7]:
image_name = "012_6_4"  # 002_1_2 # 012_6_4
mask = cv2.imread("manual_" + image_name + ".png", 0)
pred = cv2.imread("result_" + image_name + ".png", 0)
# start_time = time.time()
# f_score = get_figure_of_merit(pred, mask)
# end_time = time.time()
# print("The f_score is {}".format(f_score))
# print("The duration is {}".format(end_time - start_time))
start_time = time.time()
map_score = get_map_2018kdasb(pred, mask)
end_time = time.time()
print("The map_score is {}".format(map_score))
print("The duration is {}".format(end_time - start_time))

The map_score is 0.4525125698090613
The duration is 0.08244442939758301
